In [ ]:
import sys
import time

class TicTacToe:
    def __init__(self):
        self.size = 5
        self.board = [['-' for _ in range(self.size)] for _ in range(self.size)]
        self.player = 'O'
        self.opponent = 'X'
        self.empty = '-'
        self.moves = []      
        self.transposition_table = {}

    def display_board(self):
        for row in self.board:
            print(' '.join(row))

    def is_valid(self, x, y):
        return 0 <= x < self.size and 0 <= y < self.size

    def count_lines(self, player):
        count, potential_count = 0, 0 
        for i in range(self.size):
            for j in range(self.size):
                if self.board[i][j] in [player, self.empty]:
                    for dx, dy in [(0, 1), (1, 0), (1, 1), (1, -1)]:
                        if self.is_valid(i + 2 * dx, j + 2 * dy):
                            line = [self.board[i + k * dx][j + k * dy] for k in range(3)]
                            count += line.count(player) == 3
                            potential_count += line.count(player) == 2 and line.count(self.empty) == 1
        return count, potential_count

    def get_potential_weight(self):
        if len(self.moves) <= 2:
            return 0.5
        elif len(self.moves) <= 6:
            return 0.3
        elif len(self.moves) <= 10:
            return 0.1
        else:
            return 0
            
    def evaluate(self):
        player_count, player_potential = self.count_lines(self.player)
        opponent_count, opponent_potential = self.count_lines(self.opponent)
        potential_weight = self.get_potential_weight()

        defensive_weight = 0
        if len(self.moves) % 2 == 1:
            defensive_weight = 0.1

        return (player_count + potential_weight * player_potential) - \
            (opponent_count + (potential_weight + defensive_weight) * opponent_potential)

    def display_scores(self):
        actual_player_score, potential_player_score = self.count_lines(self.player)
        actual_opponent_score, potential_opponent_score = self.count_lines(self.opponent)
        potential_weight = self.get_potential_weight() + 0.2
        total_player_score = actual_player_score + potential_weight * potential_player_score
        total_opponent_score = actual_opponent_score + potential_weight * potential_opponent_score
        print(f"Actual Score: Computer({self.player}) {actual_player_score:.1f} - You({self.opponent}) {actual_opponent_score:.1f}")
        print(f"Actual + Potential: Computer({self.player}) {total_player_score:.1f} - You({self.opponent}) {total_opponent_score:.1f}\n")

    def set_search_depth(self):
        # Dynamically adjust the search depth
        global MAX_DEPTH
        if len(self.moves) == 2:
            MAX_DEPTH = 2
        elif len(self.moves) <= 10:
            MAX_DEPTH = 4
        else:
            MAX_DEPTH = 15

    def minimax(self, depth, is_max, alpha, beta):
        board_tuple = tuple(tuple(row) for row in self.board)
        
        if board_tuple in self.transposition_table:
            return self.transposition_table[board_tuple]

        if all(all(cell != self.empty for cell in row) for row in self.board) \
            or depth == MAX_DEPTH:
            return self.evaluate()
        
        if is_max:
            best = -sys.maxsize
            for i in range(self.size):
                for j in range(self.size):
                    if self.board[i][j] == self.empty:
                        self.board[i][j] = self.player
                        best = max(best, self.minimax(depth + 1, not is_max, alpha, beta))
                        alpha = max(alpha, best)
                        self.board[i][j] = self.empty
                        if beta <= alpha:
                            break
            self.transposition_table[board_tuple] = best
            return best
        else:
            best = sys.maxsize
            for i in range(self.size):
                for j in range(self.size):
                    if self.board[i][j] == self.empty:
                        self.board[i][j] = self.opponent
                        best = min(best, self.minimax(depth + 1, not is_max, alpha, beta))
                        beta = min(beta, best)
                        self.board[i][j] = self.empty
                        if beta <= alpha:
                            break
            self.transposition_table[board_tuple] = best
            return best

    def find_best_move(self):
        self.set_search_depth()
        start_time = time.time()

        best_val = -sys.maxsize
        best_move = (-1, -1)
        for i in range(self.size):
            for j in range(self.size):
                if self.board[i][j] == self.empty:
                    self.board[i][j] = self.player
                    move_val = self.minimax(0, False, -sys.maxsize, sys.maxsize)
                    self.board[i][j] = self.empty
                    if move_val > best_val:
                        best_move = (i, j)
                        best_val = move_val

        end_time = time.time()
        elapsed_time = end_time - start_time  
        print(f"{len(self.moves) + 1}.")
        print(f"Computer thought for {elapsed_time:.2f} seconds.") 

        return best_move

    def play(self):
        continue_game = True

        while continue_game:
            self.display_board()
            self.display_scores()

            if all(all(cell != self.empty for cell in row) for row in self.board):
                player_score = self.count_lines(self.player)
                opponent_score = self.count_lines(self.opponent)
                if player_score > opponent_score:
                    print(f"Computer wins! {player_score[0]}:{opponent_score[0]}")
                elif opponent_score > player_score:
                    print(f"You win! {opponent_score[0]}:{player_score[0]}")
                else:
                    print(f"It's a draw! {player_score[0]}:{opponent_score[0]}")
                return

            if len(self.moves) % 2 == int(start):   # Human Player's turn
                while True:
                    try:
                        move_input = input("Enter your move (row col), 'quit' to exit, or 0 to repeat last output: ")   
                        if move_input.lower() in ["quit", "exit"]:
                            print("Exiting game...")
                            continue_game = False
                            break                
                        if move_input == "0":
                            continue

                        row, col = map(int, move_input.split())
                        row -= 1
                        col -= 1

                        if 0 <= row < self.size and 0 <= col < self.size and self.board[row][col] == self.empty:
                            self.board[row][col] = self.opponent
                            self.moves.append((row, col))
                            print(f"{len(self.moves)}.")
                            break

                    except ValueError:
                        print("Invalid move, please try again")

            else:  # Computer's turn
                print("Computer's turn...", "\n")
                if len(self.moves) == 0:
                    print(f"{len(self.moves) + 1}.")
                    self.board[2][2] = self.player
                    self.moves.append((2, 2))
                else:    
                    best_move = self.find_best_move()
                    self.board[best_move[0]][best_move[1]] = self.player
                    self.moves.append(best_move)

            # After each move, check if the board is full
            if all(all(cell != self.empty for cell in row) for row in self.board):
                player_score = self.count_lines(self.player)
                opponent_score = self.count_lines(self.opponent)
                
                self.display_board()
                self.display_scores()                
                
                if player_score > opponent_score:
                    print(f"Computer wins! {player_score[0]}:{opponent_score[0]}")
                elif opponent_score > player_score:
                    print(f"You win! {opponent_score[0]}:{player_score[0]}")
                else:
                    print("It's a draw!")
                return

while True:
    try:
        start = int(input("Enter 0 to go first, 1 to go after:"))
        if start == 0 or start == 1:
            break
    except ValueError:
        print("please enter number 0 or 1")
        
game = TicTacToe()
game.play()